### Years 1 and Years 2 NAO skill ###

Looking at the skill of individual models for the NAO, years 1 and years 2.

In [ ]:
%matplotlib inline
%autosave 60
%load_ext autoreload
%autoreload 2

import os
import pathlib
import sys

import dask_gateway

import dask.array as da
import numpy as np
import time

In [ ]:
# Import the dictionaries
sys.path.append('/home/users/benhutch/skill-maps/')

# Import the dictionaries
import dictionaries as dicts

# Import the functions
from years_1_2_model_skill_xr_functions import *

In [ ]:
# Create a list of the models
models = ["BCC-CSM2-MR"]

In [ ]:
# Test the function for loading the data
azores_winter1 = load_model_data_xarray(model_variable="psl",
                                        model="HadGEM3-GC31-MM",
                                        experiment="dcppA-hindcast",
                                        start_year=1960,
                                        end_year=2018,
                                        grid=dicts.azores_grid_corrected,
                                        first_fcst_year=1960,
                                        last_fcst_year=1960,
                                        start_month=1,
                                        end_month=12,)

iceland_winter1 = load_model_data_xarray(model_variable="psl",
                                            model="HadGEM3-GC31-MM",
                                            experiment="dcppA-hindcast",
                                            start_year=1960,
                                            end_year=2018,
                                            grid=dicts.iceland_grid_corrected,
                                            first_fcst_year=1960,
                                            last_fcst_year=1960,
                                            start_month=1,
                                            end_month=12,)

In [ ]:
azores_winter2 = load_model_data_xarray(model_variable="psl",
                                        model="HadGEM3-GC31-MM",
                                        experiment="dcppA-hindcast",
                                        start_year=1960,
                                        end_year=2018,
                                        grid=dicts.azores_grid_corrected,
                                        first_fcst_year=1961,
                                        last_fcst_year=1961,
                                        start_month=1,
                                        end_month=12,)

iceland_winter2 = load_model_data_xarray(model_variable="psl",
                                            model="HadGEM3-GC31-MM",
                                            experiment="dcppA-hindcast",
                                            start_year=1960,
                                            end_year=2018,
                                            grid=dicts.iceland_grid_corrected,
                                            first_fcst_year=1961,
                                            last_fcst_year=1961,
                                            start_month=1,
                                            end_month=12,)

In [ ]:
azores_winter1

In [ ]:
azores_winter2

In [ ]:
# Calculate and remove the climatology
# Take the mean over ensemble members and time
azores_winter1_clim = azores_winter1.mean(dim=["ensemble_member", "time"])

# Calculate the anomaly
azores_winter1_anom = azores_winter1 - azores_winter1_clim

# Take the mean over ensemble members and time
iceland_winter1_clim = iceland_winter1.mean(dim=["ensemble_member", "time"])

# Calculate the anomaly
iceland_winter1_anom = iceland_winter1 - iceland_winter1_clim

In [ ]:
# Calculate and remove the climatology
# Take the mean over ensemble members and time
azores_winter2_clim = azores_winter2.mean(dim=["ensemble_member", "time"])

# Calculate the anomaly
azores_winter2_anom = azores_winter2 - azores_winter2_clim

# Take the mean over ensemble members and time
iceland_winter2_clim = iceland_winter2.mean(dim=["ensemble_member", "time"])

# Calculate the anomaly
iceland_winter2_anom = iceland_winter2 - iceland_winter2_clim

In [ ]:
# Calculate the nao
nao_winter1 = azores_winter1_anom - iceland_winter1_anom

In [ ]:
# Calculate the nao
nao_winter2 = azores_winter2_anom - iceland_winter2_anom

In [ ]:
# Load the observations
regrid_obs_path = "/home/users/benhutch/ERA5/global_regrid_sel_region.nc"

# Open the obs
obs = xr.open_mfdataset(regrid_obs_path,
                        combine="by_coords",
                        parallel=True,
                        chunks={"time": 10, "lat": 50, "lon": 50})["msl"]

# Combine the first two expver variables
# Combine the first two expver variables
obs = obs.sel(expver=1).combine_first(obs.sel(expver=5))

In [ ]:
# Set up the lats and lons
ilat1, ilat2 = dicts.iceland_grid_corrected["lat1"], dicts.iceland_grid_corrected["lat2"]

ilon1, ilon2 = dicts.iceland_grid_corrected["lon1"], dicts.iceland_grid_corrected["lon2"]

# For the azores
alat1, alat2 = dicts.azores_grid_corrected["lat1"], dicts.azores_grid_corrected["lat2"]

alon1, alon2 = dicts.azores_grid_corrected["lon1"], dicts.azores_grid_corrected["lon2"]

In [ ]:
# Select the region
obs_iceland = obs.sel(lat=slice(ilat1, ilat2),
                       lon=slice(ilon1, ilon2)).mean(dim=["lat", "lon"])

# Select the region
obs_azores = obs.sel(lat=slice(alat1, alat2),
                      lon=slice(alon1, alon2)).mean(dim=["lat", "lon"])

# Select the months
months = [12, 1, 2, 3] # DJFM

# Select the years
years = np.arange(1960, 2019 + 1)

# Constrain the obs to the same years
obs_iceland = obs_iceland.sel(time=obs_iceland["time.year"].isin(years))

# Constrain the obs to the same years
obs_azores = obs_azores.sel(time=obs_azores["time.year"].isin(years))

# Constrain the obs to the same months
obs_iceland = obs_iceland.sel(time=obs_iceland["time.month"].isin(months))

# Constrain the obs to the same months
obs_azores = obs_azores.sel(time=obs_azores["time.month"].isin(months))

In [ ]:
# Look at the obs
obs_iceland

In [ ]:
# Look at the obs
obs_azores

In [ ]:
# Remove the initial and final time points
obs_iceland = obs_iceland.isel(time=slice(3, -1))

# Remove the initial and final time points
obs_azores = obs_azores.isel(time=slice(3, -1))

In [ ]:
# Calculate the climatology
obs_iceland_clim = obs_iceland.mean(dim="time")

# Calculate the anomaly
obs_iceland_anom = obs_iceland - obs_iceland_clim

# Calculate the climatology
obs_azores_clim = obs_azores.mean(dim="time")

# Calculate the anomaly
obs_azores_anom = obs_azores - obs_azores_clim

In [ ]:
# Because DJFM
# Shift back by -3 and then calculate the annual mean anomalies
obs_iceland_anom = obs_iceland_anom.shift(time=-3).resample(time="Y").mean("time")

# Because DJFM
# Shift back by -3 and then calculate the annual mean anomalies
obs_azores_anom = obs_azores_anom.shift(time=-3).resample(time="Y").mean("time")

In [ ]:
# Reset the time axis to be july for each year
obs_iceland_anom["time"] = obs_iceland_anom["time.year"]

# Reset the time axis to be july for each year
obs_azores_anom["time"] = obs_azores_anom["time.year"]

In [ ]:
# Calculate the nao
nao_obs = obs_azores_anom - obs_iceland_anom

In [ ]:
# Have a look at the obs
nao_obs

In [ ]:
# Set the nao_winter1 time axis to be the same as the obs
nao_winter1["time"] = nao_winter1["time.year"]

# Set the nao_winter2 time axis to be the same as the obs
nao_winter2["time"] = nao_winter2["time.year"]

In [ ]:
# Divide the values by 100
nao_winter1 = nao_winter1 / 100

# Divide the values by 100
nao_winter2 = nao_winter2 / 100

# Divide the obs by 100
nao_obs = nao_obs / 100

In [ ]:
# Calculate the ensemble mean
ensemble_mean = nao_winter1.psl.mean(dim='ensemble_member')

# Plot the individual ensemble members as points
nao_winter1.psl.plot.line(x="time", hue="ensemble_member", marker='o',
                          linestyle='', color='grey', alpha=0.5, add_legend=False)

# Plot the ensemble mean as a line
ensemble_mean.plot.line(x="time", add_legend=False, color='red',
                        label="dcppA")

# Plot the obs
nao_obs.plot.line(x="time", color='k', add_legend=False,
                  label="ERA5")

# Calculate the correlation between the ensemble mean and the obs
correlation = xr.corr(nao_obs, ensemble_mean)

# Add the correlation to the plot
plt.text(0.05, 0.95, f"Correlation: {correlation.values:.2f}",
         horizontalalignment='left',
         verticalalignment='top',
         transform=plt.gca().transAxes,
         bbox=dict(facecolor='white', alpha=0.5))

# Include a horizontal line at 0
plt.axhline(0, color='k', linestyle='--')

# Add the legend
plt.legend()

In [ ]:
years = np.arange(1982, 2017 + 1)

# Constrain nao_winter2 to the same years
nao_winter2 = nao_winter2.sel(time=nao_winter2["time"].isin(years))

# Constrain nao_obs to the same years
nao_obs = nao_obs.sel(time=nao_obs["time"].isin(years))

In [ ]:
# Do the same for the second year
# Calculate the ensemble mean
ensemble_mean = nao_winter2.psl.mean(dim='ensemble_member')

# # Extract the time axis from the model data
# time = nao_winter2["time"]

# # Make sure that the obs has the same time axis
# nao_obs = nao_obs.sel(time=time)

# # Extract the time axis from the obs data
# time = nao_obs["time"]

# # Make sure that the model data has the same time axis
# nao_winter2 = nao_winter2.sel(time=time)

# Plot the individual ensemble members as points
nao_winter2.psl.plot.line(x="time", hue="ensemble_member", marker='o',
                          linestyle='', color='grey', alpha=0.5, add_legend=False)

# Plot the ensemble mean as a line
ensemble_mean.plot.line(x="time", add_legend=False, color='red',
                        label="dcppA")

# Plot the obs
nao_obs.plot.line(x="time", color='k', add_legend=False,
                  label="ERA5")

# Calculate the correlation between the ensemble mean and the obs
correlation = xr.corr(nao_obs, ensemble_mean)

# Add the correlation to the plot
plt.text(0.05, 0.95, f"Correlation: {correlation.values:.2f}",
         horizontalalignment='left',
         verticalalignment='top',
         transform=plt.gca().transAxes,
         bbox=dict(facecolor='white', alpha=0.5))

# Include a horizontal line at 0
plt.axhline(0, color='k', linestyle='--')

# Add the legend
plt.legend()

In [ ]:
# Plot the individual ensemble members as points
nao_winter1.psl.plot.line(x="time", hue="ensemble_member", marker='o',
                          linestyle='', color='grey', alpha=0.5, add_legend=False)